# CGMF Summary File Generator

In [1]:
import numpy as np
import scipy as sp
import math
import os
from statistics import *
from datetime import datetime

In [2]:
def fileInt(name):
    if (name.find("YIELDS") != -1):
        return int(name[name.find("YIELDS")+6:])
    else:
        return 1000000

def makeSummaryFile(filepath, summaryFileName, ParamName, ParamArr, option = 'meanvals'):
    
    # the intent of the variable "option" is to allow for other types of summary files to be generated
    # for example a P(nu) summary file
    
    
    # check the filepath : the path for the outputfiles as well as the summary file
    if not os.path.isdir(filepath):
        raise ValueError('You do not have the directory {} in your current working directory'.format(filepath))

    print(sorted(os.listdir(filepath), key=lambda x: fileInt(x)))
    
    
    # ** OVERWRITE ** if there exists the file summaryFileName
    summary_file_path = os.path.join(filepath, summaryFileName)
    if os.path.isfile(summary_file_path):
        f = open(summary_file_path, 'w')
        f.close()
    
    
    
    # write the header according to the option
    if (option == 'meanvals'):
        f = open(summary_file_path, 'w')
        f.write("#CGMF SUMMARY FILE\n")
        f.write("#Generated: " + str(datetime.now().strftime("%Y-%m-%d %H:%M")) + "\n")
        f.write("#Number of CGMF Output Files: {0:2d}\n#\n".format(len(os.listdir(filepath))))
        f.write("#File\t\t\t\t\t\t\t\t\t# Hist\t{}\t\t<TXE>\t1sig\t\t<TKE>\t1sig\t\t<J>\t\t1sig\t\t<nn>\tn-mom2\tn-mom3\t<ng>\tg-mom2\tg-mom3\t<En>CM\t1sig\t\t<Eg>CM\t1sig\t\t<En>Lab\t1sig\t\t<Eg>Lab\t1sig\n".format(ParamName))
        f.write("#-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\n")
        f.close()
        
    # gather the data file by file and write to file according to option
    
    # if user desires, a specific number of histories can be read, otherwise the max is infinite
    MaxHistories = 100000

    # total number of files to be read
    n_files = len(os.listdir(filepath))

    # initialize the index of the files to 0
    file_index = 0

    # From filepath, read the files individually
    # reads all files in order according to listdir

    for filename in sorted(os.listdir(filepath), key=lambda x: fileInt(x)):
        
        #skip the iteration of the summaryfile
        if (filename == summaryFileName):
            continue

        file_index += 1
        file_loc = os.path.join(filepath,filename)
        working_file = open(file_loc)

        # read first line -----------------------------------------------------------------------------------------------
        line = working_file.readline()
        if (line.find("#")!=0):
            sys.exit("ERROR: FIRST LINE OF OUTPUT FILE SHOULD CONTAIN '#'")
        else:    
            data          = line[1:].split()
            ZAIDc         = int(data[0])
            Zc            = int(ZAIDc/1000)
            Ac            = int(ZAIDc)-1000*Zc
            Asym          = int(float(Ac)/2.0)
            Einc          = float(data[1])
            nevents       = int(data[2])
            alpha         = float(data[3])

        # read remainder -----------------------------------------------------------------------------------------------
        nfragments = 0

        # re-initialize all arrays
        # Line 1 Arrays of interest

        Ul, Uh, TXE      = [], [], []
        Jl, Jh, J        = [], [], []
        KEl, KEh, TKE    = [], [], []
        nnl, nnh, nnt    = [], [], []
        ngl, ngh, ngt    = [], [], []

        # Direction/momentum vectors are arranged with CoM first, then Lab
        # Fragment momentum arrays - Line 2
        #Pcml, Pcmh       = [], []
        #Plabl, Plabh     = [], []

        # For now, just save the resulting particle energies
        # Neutron Energies - Line 3
        Encml, Encmh     = [], []
        Enlabl, Enlabh   = [], []

        # Gamma Energies - Line 4
        Egcml, Egcmh     = [], []
        Eglabl, Eglabh   = [], []

        while True:
            line = working_file.readline()
            if (len(line)==0):
                break
            #LINE ONE OF FISSION EVENT (A, Z, U, J, P, KE, nn, ng, nIC)
            nfragments+=1
            tempdata1 = line.split()
            if (nfragments % 2 == 1): # LIGHT FRAGMENT
                Ul.append(float(tempdata1[2]))
                Jl.append(float(tempdata1[3]))
                KEl.append(float(tempdata1[5]))
                nnl.append(int(tempdata1[6]))
                ngl.append(int(tempdata1[7]))

            else: # HEAVY FRAGMENT
                Uh.append(float(tempdata1[2]))
                Jh.append(float(tempdata1[3]))
                KEh.append(float(tempdata1[5]))
                nnh.append(int(tempdata1[6]))
                ngh.append(int(tempdata1[7]))


            #LINE TWO OF FISSION EVENT (momentum vector in Lab and center of mass of FRAGMENT)
            line = working_file.readline()
            tempdata2 = line.split()
            # for now, don't worry about line 2

            #LINE THREE OF FISSION EVENT (Neutron direction cosines and energies, Lab and center of mass respectively)
            if (int(tempdata1[6]) != 0):
                line = working_file.readline()
                tempdata3 = line.split()
                if (nfragments % 2 == 1): # LIGHT FRAGMENT
                    for i in range(int(tempdata1[6])):
                        Encml.append(float(tempdata3[4*i + 7]))
                        Enlabl.append(float(tempdata3[8*i + 7]))
                else: # HEAVY FRAGMENT
                    for i in range(int(tempdata1[6])):
                        Encmh.append(float(tempdata3[4*i + 7]))
                        Enlabh.append(float(tempdata3[8*i + 7]))


            #LINE FOUR OF FISSION EVENT (Gamma direction cosines and energies, Lab and center of mass respectively)
            if (int(tempdata1[7]) != 0):
                line = working_file.readline()
                tempdata4 = line.split()
                if (nfragments % 2 == 1): # LIGHT FRAGMENT
                    for i in range(int(tempdata1[7])):
                        Egcml.append(float(tempdata4[4*i + 7]))
                        Eglabl.append(float(tempdata4[8*i + 7]))
                else: # HEAVY FRAGMENT
                    for i in range(int(tempdata1[7])):
                        Egcmh.append(float(tempdata4[4*i + 7]))
                        Eglabh.append(float(tempdata4[8*i + 7]))

            if (nfragments>=MaxHistories*2):
                break
        nhistories = nfragments/2

        # Concatenate arrays
        TXE    = np.sum([Ul , Uh], axis=0)
        J      = Jl + Jh
        TKE    = np.sum([KEl , KEh], axis=0)
        nnt    = np.sum([nnl , nnh], axis=0)
        nn2t   = [float(nnt[i] * (nnt[i]-1)) for i in range(len(nnt))] # 2nd factorial moment
        nn3t   = [float(nnt[i] * (nnt[i]-1) * (nnt[i]-2)) for i in range(len(nnt))] # 3rd factorial moment
        ngt    = np.sum([ngl , ngh], axis=0)
        ng2t   = [float(ngt[i] * (ngt[i]-1)) for i in range(len(ngt))]
        ng3t   = [float(ngt[i] * (ngt[i]-1) * (ngt[i]-2)) for i in range(len(ngt))]
        Encm  = Encml + Encmh
        Egcm  = Egcml + Egcmh
        Enlab = Enlabl + Enlabh
        Eglab = Eglabl + Eglabh

        # Close File that is read
        working_file.close()

        # Open Summary File

        summary = open(summary_file_path,"a")
        
        # tabsstr aligns the second column
        tabsstr = ''
        for i in range(int(math.ceil((44.0-float(len(filename)))/8.0))):
            tabsstr = tabsstr + "\t"


        if(option == 'meanvals'):
            concstr = filename + tabsstr + '{:d}\t{:0.3f}\t{:0.3f}\t{:0.3E}\t{:0.3f}\t{:0.3E}\t{:0.3f}\t{:0.3E}\t\
        {:0.3f}\t{:0.3f}\t{:0.3f}\t{:0.3f}\t{:0.3f}\t{:0.2f}\t{:0.3f}\t{:0.3E}\t{:0.3f}\t{:0.3E}\t{:0.3f}\t{:0.3E}\t{:0.3f}\t{:0.3E}\n'.format(
                            int(nhistories), ParamArr[file_index -1],
                            mean(TXE), pstdev(TXE),  
                            mean(TKE), pstdev(TKE), mean(J),
                            pstdev(J), mean(nnt.astype(float)), mean(nn2t),
                            mean(nn3t), mean(ngt.astype(float)), mean(ng2t), mean(ng3t), mean(Encm), pstdev(Encm), 
                            mean(Egcm), pstdev(Egcm), mean(Enlab), pstdev(Enlab), mean(Eglab), pstdev(Eglab))

            summary.write(concstr)
            
        summary.close()
        print( "{:0.1f}% complete".format(file_index/(n_files-1)*100))
    
    

In [3]:
# set up the parameters dictionary

w = [0.8128359323532158, 0.1734091083059585, 0.013754959340825731]
dmin= [14.4946848, 10.8924571, 16.7133975]
dmax =[18.2042691, 17.322745, 20.4274353]
ddec =[0.446051597, 0.134703577, 0.165676002]
Abar =[144.542941, 136.328441, 127.53033]
sigA =[6.11049848, 3.69590546, 18.225735]

PARAMSdict   = {}
PARAMSdict['w0'] = np.linspace(0.75, 0.9862, 10)
PARAMSdict['w0'] = np.insert(PARAMSdict['w0'], 0, w[0])

PARAMSdict['w1'] = np.linspace(0.15, 0.25, 10)
PARAMSdict['w1'] = np.insert(PARAMSdict['w1'], 0, w[1])

PARAMSdict['w2'] = np.linspace(0.001, 0.07, 10)
PARAMSdict['w2'] = np.delete(PARAMSdict['w2'] , 2)
PARAMSdict['w2'] = np.insert(PARAMSdict['w2'], 0, w[2])

PARAMSdict['dmin0'] = np.linspace(10.,18.,10)
PARAMSdict['dmin0'] = np.delete(PARAMSdict['dmin0'] , 2)
PARAMSdict['dmin0'] = np.insert(PARAMSdict['dmin0'], 0, dmin[0])

PARAMSdict['dmin1'] = np.linspace(6.,16.,10)
PARAMSdict['dmin1'] = np.delete(PARAMSdict['dmin1'] , 3)
PARAMSdict['dmin1'] = np.delete(PARAMSdict['dmin1'] , 8) #for some reason file 50 is giving us issues.. just delete
PARAMSdict['dmin1'] = np.insert(PARAMSdict['dmin1'], 0, dmin[1])

PARAMSdict['dmin2'] = np.linspace(10.,20.,10)
PARAMSdict['dmin2'] = np.insert(PARAMSdict['dmin2'], 0, dmin[2])

PARAMSdict['dmax0'] = np.linspace(18.286,18.716,10)
# PARAMSdict['dmax0'] = np.delete(PARAMSdict['dmax0'] , 7)
PARAMSdict['dmax0'] = np.insert(PARAMSdict['dmax0'], 0, dmax[0])

PARAMSdict['dmax1'] = np.linspace(15.5426,17.432,10)
PARAMSdict['dmax1'] = np.delete(PARAMSdict['dmax1'], 3)
PARAMSdict['dmax1'] = np.insert(PARAMSdict['dmax1'], 0, dmax[1])

PARAMSdict['dmax2'] = np.linspace(17.,26.,10)
PARAMSdict['dmax2'] = np.insert(PARAMSdict['dmax2'], 0, dmax[2])

PARAMSdict['ddec0'] = np.linspace(0.2,0.8,10)
# PARAMSdict['ddec0'] = np.delete(PARAMSdict['ddec0'], 6)
PARAMSdict['ddec0'] = np.insert(PARAMSdict['ddec0'], 0, ddec[0])

PARAMSdict['ddec1'] = np.linspace(0.05,0.3,10)
PARAMSdict['ddec1'] = np.delete(PARAMSdict['ddec1'], 0)
PARAMSdict['ddec1'] = np.insert(PARAMSdict['ddec1'], 0, ddec[1])

PARAMSdict['ddec2'] = np.linspace(0.05,0.4,10)
PARAMSdict['ddec2'] = np.insert(PARAMSdict['ddec2'], 0, ddec[2])

PARAMSdict['Abar0'] = np.linspace(144.1,149.,10)
PARAMSdict['Abar0'] = np.insert(PARAMSdict['Abar0'], 0, Abar[0])

PARAMSdict['Abar1'] = np.linspace(128.,140.,10)
PARAMSdict['Abar1'] = np.delete(PARAMSdict['Abar1'], 7)
PARAMSdict['Abar1'] = np.delete(PARAMSdict['Abar1'], 8)
PARAMSdict['Abar1'] = np.insert(PARAMSdict['Abar1'], 0, Abar[1])

PARAMSdict['sigA0'] = np.linspace(4.0,8.0,10)
PARAMSdict['sigA0'] = np.insert(PARAMSdict['sigA0'], 0, sigA[0])

PARAMSdict['sigA1'] = np.linspace(2.5,5.5,10)
PARAMSdict['sigA1'] = np.delete(PARAMSdict['sigA1'], 2)
PARAMSdict['sigA1'] = np.insert(PARAMSdict['sigA1'], 0, sigA[1])

PARAMSdict['sigA2'] = np.linspace(12.0,30.0,10)
for i in range(5,10):
    PARAMSdict['sigA2'] = np.delete(PARAMSdict['sigA2'],5)
PARAMSdict['sigA2'] = np.insert(PARAMSdict['sigA2'], 0, sigA[2])


In [4]:
DATAPATHbase = '/home/austinlc/Documents/SensitivityData2'

for k,v in PARAMSdict.items():
    DATAPATH = os.path.join(DATAPATHbase,k)
    makeSummaryFile(DATAPATH, 'summary_{}.txt'.format(k), k, v)

['histories-vectors.CGMF.252CfYIELDS0', 'histories-vectors.CGMF.252CfYIELDS1', 'histories-vectors.CGMF.252CfYIELDS2', 'histories-vectors.CGMF.252CfYIELDS3', 'histories-vectors.CGMF.252CfYIELDS4', 'histories-vectors.CGMF.252CfYIELDS5', 'histories-vectors.CGMF.252CfYIELDS6', 'histories-vectors.CGMF.252CfYIELDS7', 'histories-vectors.CGMF.252CfYIELDS8', 'histories-vectors.CGMF.252CfYIELDS9', 'histories-vectors.CGMF.252CfYIELDS10', 'summary_w0.txt']
9.1% complete
18.2% complete
27.3% complete
36.4% complete
45.5% complete
54.5% complete
63.6% complete
72.7% complete
81.8% complete
90.9% complete
100.0% complete
['histories-vectors.CGMF.252CfYIELDS0', 'histories-vectors.CGMF.252CfYIELDS11', 'histories-vectors.CGMF.252CfYIELDS12', 'histories-vectors.CGMF.252CfYIELDS13', 'histories-vectors.CGMF.252CfYIELDS14', 'histories-vectors.CGMF.252CfYIELDS15', 'histories-vectors.CGMF.252CfYIELDS16', 'histories-vectors.CGMF.252CfYIELDS17', 'histories-vectors.CGMF.252CfYIELDS18', 'histories-vectors.CGMF.25

11.1% complete
22.2% complete
33.3% complete
44.4% complete
55.6% complete
66.7% complete
77.8% complete
88.9% complete
100.0% complete
['histories-vectors.CGMF.252CfYIELDS0', 'histories-vectors.CGMF.252CfYIELDS141', 'histories-vectors.CGMF.252CfYIELDS142', 'histories-vectors.CGMF.252CfYIELDS143', 'histories-vectors.CGMF.252CfYIELDS144', 'histories-vectors.CGMF.252CfYIELDS145', 'histories-vectors.CGMF.252CfYIELDS146', 'histories-vectors.CGMF.252CfYIELDS147', 'histories-vectors.CGMF.252CfYIELDS148', 'histories-vectors.CGMF.252CfYIELDS149', 'histories-vectors.CGMF.252CfYIELDS150', 'summary_sigA0.txt']
9.1% complete
18.2% complete
27.3% complete
36.4% complete
45.5% complete
54.5% complete
63.6% complete
72.7% complete
81.8% complete
90.9% complete
100.0% complete
['histories-vectors.CGMF.252CfYIELDS0', 'histories-vectors.CGMF.252CfYIELDS151', 'histories-vectors.CGMF.252CfYIELDS152', 'histories-vectors.CGMF.252CfYIELDS154', 'histories-vectors.CGMF.252CfYIELDS155', 'histories-vectors.CGMF.